In [1]:
import pandas
#from geosnap import Community
#from geosnap import datasets
import fiona
import geopandas
%matplotlib inline
import matplotlib.pyplot as plt
import segregation

In [2]:
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_column', 500)

In [3]:
ele = geopandas.read_file("districts_tiger/tl_2015_us_sde.shp")
# Reading in shapefiles from census/tiger

In [4]:
seco = geopandas.read_file("districts_tiger/tl_2015_us_sds.shp")

In [5]:
uni = geopandas.read_file("districts_tiger/tl_2015_us_sdu.shp")

In [6]:
ele = ele.loc[ele['STATEFP'] == '06']
# slicing out California
# elementary school districts in California

In [7]:
seco = seco.loc[seco['STATEFP'] == '06']
# secondary school districts in California

In [8]:
uni = uni.loc[uni['STATEFP'] == '06']
# unified school districts in California

In [9]:
len(ele)+len(seco)+len(uni)

986

## Combining shapefiles into single geodataframe

In [10]:
ele['origin'] = 'elementary'

In [11]:
seco['origin'] = 'secondary'

In [12]:
uni['origin'] = 'unified'

In [13]:
adistricts = uni.append([ele, seco])
#all districts

In [14]:
adistricts = adistricts.sort_values('NAME', ascending=True)

In [15]:
adistricts.head()

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
645,06,01620,0601620,ABC Unified School District,00,KG,12,G5420,None,E,38772821,556042,+33.8637711,-118.0665456,"POLYGON ((-118.10857 33.88775, -118.10843 33.8...",unified,NaN,NaN
29,06,NaN,0601650,Acalanes Union High School District,00,09,12,G5410,None,E,205137702,4413448,+37.8772150,-122.1288576,"POLYGON ((-122.26126 37.92143, -122.26009 37.9...",secondary,NaN,01650
403,06,NaN,0601680,Ackerman Elementary School District,00,KG,08,G5400,None,E,11308882,7628,+38.9366318,-121.0507823,"POLYGON ((-121.07045 38.92923, -121.07045 38.9...",elementary,01680,NaN
658,06,00001,0600001,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,555175128,182565,+34.4509057,-118.2276141,"POLYGON ((-118.38010 34.50369, -118.37724 34.5...",unified,NaN,NaN
376,06,NaN,0601710,Adelanto Elementary School District,00,KG,06,G5400,None,E,868470886,832474,+34.6306380,-117.5205628,"POLYGON ((-117.66785 34.79074, -117.66784 34.7...",elementary,01710,NaN


In [16]:
adistricts.shape

(986, 18)

In [17]:
adsgeo = adistricts['NAME']
# all districs shapfile geodataframe

In [18]:
adsgeo.unique().shape
# todo: some districts listed twice

(970,)

In [19]:
dupes = adistricts[adistricts['NAME'].isin(adistricts['NAME'].value_counts()[adistricts['NAME'].value_counts()>2].index)]
print(dupes['NAME'].unique())
# 3 Jefferson
# 3 Lakeside
# 3 Pioneer

# create new column indicating countyID

['Jefferson Elementary School District'
 'Lakeside Union Elementary School District'
 'Pioneer Union Elementary School District']


### Harmonizing data

Current Status - Average distance from Standard of students who took the Smarter Balanced summative assessment in ELA


options
questions of differences across groups? (no geography)

differences across districts (group by district)

or differences across groups in districts (interaction effects > more pronounced differences in certain districts)

visualization

what is the best indicator to use for choropleth ()

or - use state level values as benchmark for each group (how is it reflected in each district, heterogeniety in space?)

## ELA acheivement indicator data
Find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/ela18.asp)

In [16]:
replacements = dict(
    studentgroup = {'ALL':'all_students',
                        'AA':'african_american',
                        'AI':'american_indian',
                        'AS':'asian',
                        'FI':'filipino',
                        'HI':'hispanic',
                        'PI':'pacific_islander',
                        'WH':'white',
                        'MR':'multiple_races',
                        'EL':'english_learners',
                        'ELO':'english_learners_only',
                        'RFP':'rfep_only',
                        'EO':'english_only',
                        'SED':'ses_disad',
                        'SWD':'disabilities',
                        'FOS':'foster_youth',
                        'HOM':'homeless_youth',
                })

In [252]:
ela = pandas.read_csv('lcff_data/eladownload2018.csv')
ela.shape

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148838, 32)

In [253]:
ela.head(500)

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear
0,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AA,172224,NaN,-51.8,177009.0,NaN,-52.7,0.9,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AI,16662,NaN,-36.8,17234.0,NaN,-39.1,2.3,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
2,0,X,NaN,NaN,NaN,NaN,NaN,NaN,ALL,3171704,NaN,-6.0,3200441.0,NaN,-8.2,2.2,2,3,2,180,NaN,NaN,NaN,NaN,36140.0,18581.0,51.4,11330.0,31.4,6229.0,17.2,2018
3,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AS,293002,NaN,62.4,295040.0,NaN,60.5,2.0,5,3,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
4,0,X,NaN,NaN,NaN,NaN,NaN,NaN,EL,1000703,NaN,-47.1,1021418.0,NaN,-50.4,3.3,2,4,3,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
5,0,X,NaN,NaN,NaN,NaN,NaN,NaN,ELO,512680,NaN,-94.1,530456.0,NaN,-96.7,2.6,1,3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
6,0,X,NaN,NaN,NaN,NaN,NaN,NaN,EO,1795002,NaN,6.5,1804785.0,NaN,4.6,1.8,3,3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
7,0,X,NaN,NaN,NaN,NaN,NaN,NaN,FI,71139,NaN,44.0,73769.0,NaN,41.2,2.7,4,3,4,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
8,0,X,NaN,NaN,NaN,NaN,NaN,NaN,FOS,21073,NaN,-79.2,22846.0,NaN,-80.6,1.4,1,3,1,230,NaN,-0.25,-0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
9,0,X,NaN,NaN,NaN,NaN,NaN,NaN,HI,1738615,NaN,-31.3,1743912.0,NaN,-34.5,3.2,2,4,3,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


In [382]:
i = ela.loc[ela['rtype'] == 'S']
o = i = pandas.concat([i.pop(x) for x in ['schoolname', 
                                      'cds']],
                  1)
o
# had to check this school's cds code to validate with another dataset

,schoolname,cds
30,Envision Academy for Arts & Technology,1100170112607
31,Envision Academy for Arts & Technology,1100170112607
32,Envision Academy for Arts & Technology,1100170112607
33,Envision Academy for Arts & Technology,1100170112607
34,Envision Academy for Arts & Technology,1100170112607
...,...,...
148833,Wheatland Union High,58727695838305
148834,Wheatland Union High,58727695838305
148835,Wheatland Union High,58727695838305
148836,Wheatland Union High,58727695838305


In [34]:
# creating state-level ela performance variable
ela['studentgroup'].astype(str)
ela = ela.replace(replacements)
# replace codes 
j = ela.loc[ela['rtype'] == 'X']
# select by no-type (state)
k = pandas.concat([j.pop(x) for x in ['studentgroup', 
                                      'currstatus', 
                                      'priorstatus', 
                                      'change',
                                      'currdenom']],
                  1)
# slice out columns
ela_state = k.sort_values(by='currstatus', ascending=False)
ela_state
# state-level ela performance variable

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,62.4,60.5,2.0,293002
7,filipino,44.0,41.2,2.7,71139
11,multiple_races,28.6,26.3,2.3,118753
16,white,27.7,26.9,0.8,734863
6,english_only,6.5,4.6,1.8,1795002
13,rfep_only,2.2,-0.4,2.7,488023
2,all_students,-6.0,-8.2,2.2,3171704
12,pacific_islander,-21.3,-22.0,0.7,14894
9,hispanic,-31.3,-34.5,3.2,1738615
14,ses_disad,-34.7,-38.7,4.0,1982522


,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear


## Aggregation 

In [36]:
ela['cds'].astype(str)
# changing cds code to string
ela = ela.loc[ela['rtype'] == 'D']
# select district level entries
ela['cds'] = ela['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
ela['cds'].astype(str)
# changing to string
elat = set(ela['cds'])
# creating test set variable

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [37]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in ela.iterrows(): 
    # Create list for the current row 
    my_list =[rows.studentgroup, rows.currstatus, rows.priorstatus, rows.change, rows.currdenom] 
      
    # append the list to the final list 
    Row_list.append(my_list) 
  
 
Row_list

[[nan, nan, nan, 2],
 [-157.0, -186.0, 29.0, 20],
 [nan, nan, nan, 1],
 [nan, nan, nan, 8],
 [nan, nan, nan, 8],
 [nan, -182.2, nan, 8],
 [nan, nan, nan, 1],
 [nan, nan, nan, 2],
 [-151.3, nan, nan, 14],
 [nan, nan, nan, 2],
 [nan, nan, nan, 2],
 [-165.3, -177.4, 12.1, 16],
 [nan, nan, nan, 5],
 [-43.0, -48.7, 5.7, 298],
 [-14.8, 14.3, -29.1, 13],
 [36.8, 32.8, 4.0, 4550],
 [48.6, 54.0, -5.3, 1288],
 [-2.5, -1.4, -1.1, 1084],
 [-42.0, -44.7, 2.7, 643],
 [43.5, 38.3, 5.2, 3025],
 [29.8, 16.0, 13.9, 275],
 [nan, nan, nan, 9],
 [5.0, 0.9, 4.1, 720],
 [-75.5, -56.7, -18.7, 30],
 [46.6, 40.4, 6.2, 555],
 [-27.3, -49.1, 21.8, 40],
 [55.0, 55.5, -0.5, 441],
 [-13.1, -20.0, 6.9, 1491],
 [-73.1, -80.4, 7.3, 506],
 [60.8, 54.1, 6.7, 1346],
 [-9.6, 12.2, -21.8, 53],
 [nan, nan, nan, 1],
 [57.5, 64.1, -6.6, 1887],
 [73.0, 73.7, -0.8, 513],
 [31.0, 37.6, -6.6, 449],
 [-11.4, -12.5, 1.1, 186],
 [60.9, 68.0, -7.1, 1234],
 [38.0, 23.3, 14.7, 15],
 [nan, nan, nan, 1],
 [25.6, 28.9, -3.3, 325],
 [nan, n

In [29]:
#creating district-level ela performance variable (dictionary)
dfe = ela
dfe = pandas.concat([dfe.pop(x) for x in ['cds',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change',
                                          'currdenom']],
                  1)
# slice out columns

In [30]:
dfe

,cds,districtname,studentgroup,currstatus,priorstatus,change,currdenom
17,01100170000000,Alameda County Office of Education,african_american,NaN,NaN,NaN,2
18,01100170000000,Alameda County Office of Education,all_students,-157.0,-186.0,29.0,20
19,01100170000000,Alameda County Office of Education,asian,NaN,NaN,NaN,1
20,01100170000000,Alameda County Office of Education,english_learners,NaN,NaN,NaN,8
21,01100170000000,Alameda County Office of Education,english_learners_only,NaN,NaN,NaN,8
...,...,...,...,...,...,...,...
148816,58727690000000,Wheatland Union High,pacific_islander,NaN,NaN,NaN,1
148817,58727690000000,Wheatland Union High,rfep_only,NaN,NaN,NaN,2
148818,58727690000000,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6,69
148819,58727690000000,Wheatland Union High,disabilities,-150.3,NaN,NaN,18


In [74]:
len(elat)

980

## Math achievement indicator data
find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/math18.asp)

In [144]:
math = pandas.read_csv('lcff_data/mathdownload2018.csv')
math.shape

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148705, 32)

In [147]:
math['studentgroup'].astype(str)
math = math.replace(replacements)
# replace codes
p = math.loc[math['rtype'] == 'X']
# select by no-type (state)
q = pandas.concat([p.pop(x) for x in ['studentgroup', 
                                      'currstatus', 
                                      'priorstatus', 
                                      'change',
                                      'currdenom']],
                  1)
# slice out columns
math_state = q.sort_values(by='currstatus', ascending=False)
math_state

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,56.7,54.3,2.4,293068
7,filipino,13.1,10.3,2.8,71096
11,multiple_races,1.9,0.1,1.9,118473
16,white,-1.0,-1.6,0.6,733213
6,english_only,-25.2,-26.3,1.1,1790583
13,rfep_only,-29.7,-31.1,1.4,487555
2,all_students,-36.4,-37.7,1.3,3166345
12,pacific_islander,-52.0,-51.3,-0.7,14849
9,hispanic,-65.8,-67.5,1.7,1735982
14,ses_disad,-67.4,-69.7,2.3,1978909


In [146]:
ela_state

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,62.4,60.5,2.0,293002
7,filipino,44.0,41.2,2.7,71139
11,multiple_races,28.6,26.3,2.3,118753
16,white,27.7,26.9,0.8,734863
6,english_only,6.5,4.6,1.8,1795002
13,rfep_only,2.2,-0.4,2.7,488023
2,all_students,-6.0,-8.2,2.2,3171704
12,pacific_islander,-21.3,-22.0,0.7,14894
9,hispanic,-31.3,-34.5,3.2,1738615
14,ses_disad,-34.7,-38.7,4.0,1982522


In [76]:
math = math.loc[math['rtype'] == 'D']
# select district level performance
math['cds'] = math['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
math['cds'].astype(str)
# changing to string
matht = set(math['cds'])
# creating test variable

In [77]:
len(matht)

979

# LCFF funding data

[FAQ here](https://www.cde.ca.gov/fg/aa/lc/lcfffaq.asp#CALPADS)

In [370]:
lcff = pandas.read_csv('lcff_data/lcffsnapshot18an.csv')
lcff.shape

(2243, 26)

In [372]:
lcff.columns
# leading AND trailing spaces! inconsistently!! 

Index([' County Code ', 'District Code', 'School Code',
       'Local Educational Agency ', 'charter_number',
       ' TK/K-3 ADA\nTarget, B-5/B-1 ', ' 4 - 6 ADA\nTarget, B-6/B-2 ',
       ' 7 - 8 ADA\nTarget, B-7/B-3 ', ' 9 - 12 ADA\nTarget, B-8/B-4 ',
       'unduplicated_pupil_count', ' Base Grant Funding\nTarget, C-5 ',
       ' Supplemental Grant Funding\nTarget, D-7 ',
       ' Concentration Grant Funding\nTarget, E-8 ',
       ' Necessary Small Schools Allowance\nTarget, F-1 ',
       ' Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2 ',
       ' Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3 ',
       ' Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4 ',
       ' Total LCFF Target Entitlement\nTarget, G-1/F-1 ',
       ' Total LCFF Floor Entitlement\nTransition, B-13 ',
       ' Current Year Gap Funding (100%)\nTransition, C-3 ',
       ' Economic Recovery Target\nTransition, D-1 ',
 

In [373]:
lcff.columns = lcff.columns.str.strip()
# strip leading/trailing spaces

In [374]:
lcff_state_totals = lcff.tail(1)
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df

In [375]:
# Slicing by district level cds code
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# Changing to strings
lcff['School Code'] = lcff['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# Front fill school codes to == 7
lcff = lcff.loc[lcff['School Code'] == '0000000']
# select districts, district entries do not have a school level cds code
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# Combining into cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# Moving to frontuse 
lcff.drop('County Code', axis=1, inplace=True)
# lcff.drop('District Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
len(lcff)

944

In [377]:
# renaming important columns
lcff = lcff.rename(columns={'Charter Number': 'charter_number',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage',
                     'Base Grant Funding\nTarget, C-5': 'base_grant',
                     'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant',
                     'Concentration Grant Funding\nTarget, E-8': 'concentration_grant',
                     'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_grants',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_count',
                     'Necessary Small Schools Allowance\nTarget, F-1': 'nec_small_schools'
                           })

In [378]:
lcff.head()
# cds codes are now concurrent with documentation and comparable to acheivement data

,cds,District Code,Local Educational Agency,charter_number,"TK/K-3 ADA\nTarget, B-5/B-1","4 - 6 ADA\nTarget, B-6/B-2","7 - 8 ADA\nTarget, B-7/B-3","9 - 12 ADA\nTarget, B-8/B-4",unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,"Necessary Small Schools Allowance\nTarget, F-1","Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2","Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3","Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4",total_grants,"Total LCFF Floor Entitlement\nTransition, B-13","Current Year Gap Funding (100%)\nTransition, C-3","Economic Recovery Target\nTransition, D-1","Miscellaneous Adjustments\nTransition, E-1","Total Local Revenue or In-Lieu of Property Taxes\nTransition, F-2/F-7","Education Protection Account Entitlement\nTransition, F-5/F-9","Net State Aid\nTransition, F-6/F-10","Additional SA for MSA Guarantee\nTransition, H-1/H-2"
10,01611190000000,61119,Alameda Unified,NaN,"2,870.43","1,917.91","1,219.66","3,050.17",0.3486,"75,938,983","5,294,466",-,-,"665,778","258,889",-,"82,158,116","77,176,229","4,981,887",-,-,"29,277,891","14,772,333","38,107,892",-
16,01611270000000,61127,Albany City Unified,NaN,"1,060.09",806.37,554.25,"1,131.64",0.2833,"29,644,972","1,679,684",-,-,-,"27,884",-,"31,352,540","29,460,891","1,891,649",-,-,"11,005,115","5,777,202","14,570,223",-
17,01611430000000,61143,Berkeley Unified,NaN,"2,828.54","2,101.08","1,397.55","3,070.55",0.3519,"78,556,532","5,528,808",-,-,"4,293,233","991,048",-,"89,369,621","84,726,528","4,643,093","820,253",-,"43,425,650","8,875,387","37,888,837",-
19,01611500000000,61150,Castro Valley Unified,NaN,"2,617.44","2,032.50","1,400.40","2,918.48",0.2904,"74,911,585","4,350,864",-,-,-,"292,158",-,"79,554,607","74,489,641","5,064,966",-,-,"26,572,525","14,660,230","38,321,852",-
20,01611680000000,61168,Emery Unified,NaN,237.65,151.54,104.99,191.97,0.7953,"5,702,229","906,997","699,378",-,-,"18,271",-,"7,326,875","6,698,947","627,928","108,723",-,"5,582,328","137,230","1,716,040",-


In [381]:
#is this done?
ff = lcff[['Local Educational Agency', 'unduplicated_pupil_count', 'base_grant', 'supplemental_grant', 'concentration_grant', 'total_grants', 'cds' ]].copy()
ff.set_index(lcff['District Code'])

,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
District Code,,,,,,,
61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000
61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000
61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000
61168,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000
...,...,...,...,...,...,...,...
72728,Camptonville Elementary,0.6988,"420,692","58,796","31,299","537,244",58727280000000
72736,Marysville Joint Unified,0.8168,"76,193,862","12,447,028","10,164,261","100,261,913",58727360000000
72744,Plumas Lake Elementary,0.3868,"10,261,440","793,825",-,"11,108,901",58727440000000


In [131]:
lcfft = set(lcff['cds'])
# create test variable

# Free and reduced meal plan data
Used by policy makers as a stand-in for a  segregation indicator. 

Find the codebook [here](https://www.cde.ca.gov/ds/sd/sd/fsspfrpm.asp).

In [397]:
frpm = pandas.read_csv('lcff_data/frpm1819.csv', encoding = "ISO-8859-1", skiprows=1)
frpm.shape
# aggregate figures by district
# jump down to 'mealagg' cell

(10520, 28)

In [398]:
# renaming important columns
frpm = frpm.rename(columns={'Percent (%) \nEligible FRPM \n(K-12)': 'percent_eligible_k12_frpm', 
                            'Percent (%) \nEligible Free \n(K-12)': 'percent_eligible_k12_free', 
                            'Percent (%) \nEligible FRPM \n(Ages 5-17)' : 'percent_eligible_by_age',
                            'Enrollment \n(Ages 5-17)': 'enrollment_by_age',
                            'FRPM Count \n(Ages 5-17)': 'frpm_count_by_age',
                            'Free Meal \nCount \n(K-12)': 'free_count_by_age',
                            
                           })

In [399]:
# creating CDS code
frpm['County Code'] = frpm['County Code'].astype(str)
frpm['County Code'] = frpm['County Code'].apply(lambda x: '{0:0>2}'.format(x))
frpm['District Code'] = frpm['District Code'].astype(str)
frpm['School Code'] = frpm['School Code'].astype(str)
frpm['School Code'] = frpm['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# converting to strings, adding leading zeroes 
frpm['cds'] = frpm['County Code'] + frpm['District Code'] + frpm['School Code']
# combining codes

col_name1='cds'
first_col1 = frpm.pop(col_name1)
frpm.insert(0, col_name1, first_col1)
# Moving to front

frpm.drop('County Code', axis=1, inplace=True)
# frpm.drop('District Code', axis=1, inplace=True)
frpm.drop('School Code', axis=1, inplace=True)
# dropping split codes

In [400]:
len(frpm['cds'].iloc[10519])

14

In [405]:
frpm.head()

,cds,Academic Year,District Code,County Name,District Name,School Name,District Type,School Type,Educational \nOption Type,NSLP \nProvision \nStatus,Charter \nSchool \n(Y/N),Charter \nSchool \nNumber,Charter \nFunding \nType,IRC,Low Grade,High Grade,Enrollment \n(K-12),free_count_by_age,percent_eligible_k12_free,FRPM Count \n(K-12),percent_eligible_k12_frpm,enrollment_by_age,Free Meal \nCount \n(Ages 5-17),Percent (%) \nEligible Free \n(Ages 5-17),frpm_count_by_age,percent_eligible_by_age,CALPADS Fall 1 \nCertification Status
0,01100170112607,2018-2019,10017,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,County Office of Education (COE),High Schools (Public),Traditional,NaN,Yes,0811,Directly funded,Y,9,12,385,262,68.1%,306,79.5%,370,249,67.3%,291,78.6%,Y
1,01100170123968,2018-2019,10017,Alameda,Alameda County Office of Education,Community School for Creative Education,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1284,Directly funded,Y,K,8,241,118,49.0%,120,49.8%,224,113,50.4%,115,51.3%,Y
2,01100170124172,2018-2019,10017,Alameda,Alameda County Office of Education,Yu Ming Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1296,Directly funded,Y,K,8,445,58,13.0%,64,14.4%,445,58,13.0%,64,14.4%,Y
3,01100170125567,2018-2019,10017,Alameda,Alameda County Office of Education,Urban Montessori Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1383,Directly funded,Y,K,8,432,113,26.2%,131,30.3%,418,111,26.6%,129,30.9%,Y
4,01100170130401,2018-2019,10017,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,County Office of Education (COE),Juvenile Court Schools,Juvenile Court School,NaN,No,NaN,NaN,N,K,12,53,53,100.0%,53,100.0%,50,50,100.0%,50,100.0%,Y


In [402]:
# aggregate all schools into district level

frpm['enrollment_by_age'] = frpm['enrollment_by_age'].str.replace(',', '').astype(int)
frpm['frpm_count_by_age'] = frpm['frpm_count_by_age'].str.replace(',', '').astype(int)
frpm['free_count_by_age'] = frpm['free_count_by_age'].str.replace(',', '').astype(int)
# striping commas, converting to int 

#If nothing is int (and everything is object), all columns live through the groupby
# if we convert to int before groupby (to make .sum() work), only int lives through groupby

mealagg = frpm.groupby(by='District Code').sum()
# aggregating
mealagg['pct_frpm_by_age'] = mealagg.frpm_count_by_age / mealagg.enrollment_by_age
mealagg['pct_free_by_age'] = mealagg.free_count_by_age / mealagg.enrollment_by_age
# inferring more convinient than copying lul

In [406]:
mealagg.head()
# why do we lose columns in the groupby operation? dtypes object preventing groupby?
# are there ways to validiate percentages? groupby.multiply?
# we also want: county name, district name. 

,free_count_by_age,enrollment_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age
District Code,,,,,
10017,1857,3275,1962,0.599084,0.567023
10033,93,189,96,0.507937,0.492063
10041,592,1245,650,0.522088,0.475502
10058,230,474,260,0.548523,0.485232
10066,31,30,27,0.900000,1.033333


In [132]:
frpmt = set(frpm['cds'])
len(frpmt)

10520

### frpm data do not have district level data, but can aggregate and infer code (county + district code +'0000000')

## Match LCFF data to performance data by cds code

In [20]:
# repeat for perfomance data
# loop over both sets of names to check which one's match <-- YOU ARE HERE
# reverse loop to re-verify, see which ones are missing
# 2 lists of 
# create list of problem children

In [62]:
s1 = lcfft.intersection(elat)
len(s1)

929

In [83]:
s2 = lcfft.intersection(matht)
len(s2)

928

In [87]:
s1.difference(s2)

{'47703180000000'}

In [89]:
ela.loc[ela['cds'] == '47703180000000'].districtname.unique()
# No math performance for this district.

array(['Gazelle Union Elementary'], dtype=object)

In [91]:
matches = s1.intersection(s2)
len(matches)

928

In [81]:
# toy
x = {"apple", "banana", "cherry"}
y = {"google", "microsoft", "apple"}
z = x.difference(y)
print(z)

{'banana', 'cherry'}


In [98]:
d1 = elat.difference(lcfft)
len(d1)

51

In [99]:
d2 = lcfft.difference(elat)
len(d2)

15

In [100]:
f1 = matht.difference(lcfft)
len(f1)

51

In [101]:
f2 = lcfft.difference(matht)
len(f2)

16

inherent problems in matches - accept as structural problems in data gathering and storage (opacity)

In [ ]:
# frpm groupby district 
# 1) remove %
# 2) convert to float
# 3) groupby 
# 4) compare with dissimilarity index to validate proxy <--- YOU ARE HERE

In [ ]:
frpm['percent_eligible_k12_free'] = frpm['percent_eligible_k12_free'].str.strip('%')
frpm['percent_eligible_k12_free'].astype(float)

In [ ]:
frpm['percent_eligible_k12_frpm'] = frpm['percent_eligible_k12_frpm'].str.strip('%')
frpm['percent_eligible_k12_frpm'].astype(float)

In [ ]:
frpm.head()

In [ ]:
funding_lookup

In [ ]:
frmp['District Code']

In [ ]:
good = [code for code in funding_lookup if code in frmp['District Code']]

In [ ]:
good[-10:]

In [ ]:
for code in funding_lookup:
    if code in frmp['District Code']:
        print(code)

In [ ]:
frmp['Percent (%) \nEligible FRPM \n(K-12)']
# these are by school only.. will need to do an average to get a figure to represent the county, or do something else?

# Census Data

#### SCAG counties and their FIPS codes:
<br> Los Angeles (06037)<br> Orange (06059)<br> Riverside (06065)<br> San Bernardino (06071)<br> San Diego (06073)<br>Imperial (06025)<br> Ventura (06111)

In [ ]:
cacom = Community.from_census(state_fips='06')

In [ ]:
scag = Community.from_census(county_fips=['06025', '06037', '06059', '06071', '06111', '06073', '06065'])

In [ ]:
rivco = Community.from_census(county_fips=['06065'])

In [ ]:
orangeco = Community.from_census(county_fips=['06059'])

In [ ]:
laco = Community.from_census(county_fips=['06037'])

In [ ]:
sbco = Community.from_census(county_fips=['06071'])

In [ ]:
sdco = Community.from_census(county_fips=['06073'])

In [ ]:
f, ax = plt.subplots(1, figsize=(30, 30))
ax = rivco.gdf[rivco.gdf.year==2010].plot('p_poverty_rate_over_65', axes=ax)
plt.show()

# change gdf, variable, and year

### This 'plot' function will produce three maps, one from each of the census data years. We can use it to pass in communities and a specific variable to provide a preliminary view of changes over 30 years.

In [ ]:
def plot(community, column):
    fig, axs = plt.subplots(1,3, figsize=(20,20))
    axs=axs.flatten()
    
    community.gdf[community.gdf.year==1990].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[0])
    axs[0].axis('off')
    axs[0].set_title('1990')

    community.gdf[community.gdf.year==2000].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[1])
    axs[1].axis('off')
    axs[1].set_title('2000')

    community.gdf[community.gdf.year==2010].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[2])
    axs[2].axis('off')
    axs[2].set_title('2010')

In [ ]:
datasets.codebook()

### Variables of interest from census:
#### economic indicators
median_household_income <br>
median_home_value OR median_contract_rent<br>
median_income_whitehh<br>
median_income_blackhh<br>
median_income_hispanichh<br>
n_edu_college_greater - persons with at least 4-year degree (also available as %)<br>
n_edu_hs_less - persons with less than high school (also available as %)<br>

#### demographic indicators
n_recent_immigrant_pop (within the last 10 years)<br>
n_limited_english OR n_other_language<br>
n_hispanic_persons<br>
n_black_persons<br>
n_white_persons

In [ ]:
from segregation.aspatial import GiniSeg
index = GiniSeg(scag.gdf, 'n_mexican_pop', 'n_total_pop')
type(index)

# MERGE SCHOOL DISTRICTS WITH CENSUS DATA

## Alternative community construction

Can also construct communities from longitudinal tract database (ltdb)

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Riverside')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Orange')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('San')]

In [ ]:
LA = Community.from_ltdb(msa_fips='31080')

In [ ]:
SD = Community.from_ltdb(msa_fips='41740')

In [ ]:
riverside = Community.from_ltdb(msa_fips='40140')